In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
import datetime

In [5]:
def encode_pop(row_val):
    if(row_val=="CEU"):
        return 0
    elif(row_val=="YRI"):
        return 1
    elif(row_val=="CHB"):
        return 2

In [6]:
raw_file = pd.read_csv('ceu_yri_chb_A.raw', sep=" ",nrows=310)
raw_file.head(3)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,rs12565286_C,rs12082473_A,rs3094315_G,rs2286139_C,...,rs5770820_A,rs9628187_T,rs6010063_G,rs715586_T,rs8137951_A,rs3810648_G,rs3865766_C,rs2238837_C,rs34726907_T,rs28729663_A
0,NA06984,NA06984,0,0,0,-9,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,NA06985,NA06985,0,0,0,-9,0,0,0,0,...,1,0,2,0,2,1,0,1,1,1
2,NA06986,NA06986,0,0,0,-9,0,0,0,0,...,0,1,1,0,1,0,2,0,0,0


In [7]:
raw_file = raw_file.drop(raw_file.columns[[0,1,2,3,4,5]],axis=1)
raw_file.head(3)

,rs12565286_C,rs12082473_A,rs3094315_G,rs2286139_C,rs11240776_G,rs2980319_A,rs2980300_T,rs2905036_C,rs11240777_A,rs4245756_T,...,rs5770820_A,rs9628187_T,rs6010063_G,rs715586_T,rs8137951_A,rs3810648_G,rs3865766_C,rs2238837_C,rs34726907_T,rs28729663_A
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,2,1,0,1,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,2,0,0,0


In [8]:
snps = list(raw_file.columns.values)

In [9]:
pops = pd.read_csv('pop_names', sep=" ",nrows=310)

In [10]:
pops = pops.drop('IID',axis=1)

In [11]:
pops['pop1'] = pops['pop'].apply(encode_pop)

In [12]:
X = raw_file.as_matrix(columns=None)

In [13]:
y = np.array(pops['pop1'])

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)

In [15]:
print("Number of Samples used for Training:",len(X_train))

Number of Samples used for Training: 248


In [16]:
tstart = datetime.datetime.now()
final_model = svm.SVC(C=1,kernel="linear",decision_function_shape='ovo')
final_model.fit(X_train,Y_train)
tfinish = datetime.datetime.now()
texecute = tfinish - tstart
print(texecute)

0:01:47.767000


In [17]:
final_model.classes_

array([0, 1, 2], dtype=int64)

In [18]:
len(final_model.support_vectors_)

248

In [19]:
weights = final_model.coef_

In [20]:
weights_model0 = np.array(weights[0])
weights_model1 = np.array(weights[1])
weights_model2 = np.array(weights[2])
weights_model0 = np.abs(weights_model0)
weights_model1 = np.abs(weights_model1)
weights_model2 = np.abs(weights_model2)

In [21]:
#def getImportance(score1,score2,score3):
    #if(score1>0 and score2>0 and score3>0):
        #return "YES"
    #else:
        #return "NO"

In [22]:
#weights_df = pd.DataFrame()

In [23]:
#weights_df["SNPS"] = snps
#weights_df["MODEL_1"] = weights_model0
#weights_df["MODEL_2"] = weights_model1
#weights_df["MODEL_3"] = weights_model2

In [24]:
#weights_df["IMPORTANCE"] = weights_df.apply(lambda row: getImportance(row["MODEL_1"],row["MODEL_2"],row["MODEL_3"]), axis=1)

In [25]:
#important_snps_df = weights_df[weights_df["IMPORTANCE"]=="YES"]

In [26]:
model1_snps_dict,model2_snps_dict,model3_snps_dict = {},{},{}

In [27]:
i=0
for i in range(len(snps)):
    model1_snps_dict[snps[i]] = weights_model0[i]
    model2_snps_dict[snps[i]] = weights_model1[i]
    model3_snps_dict[snps[i]] = weights_model2[i]

In [33]:
len(model3_snps_dict)

857112

In [28]:
#sorted_model1_snps = sorted(model1_snps_dict, key=lambda x: model1_snps_dict[x], reverse=True)
#sorted_model2_snps = sorted(model2_snps_dict, key=lambda x: model2_snps_dict[x], reverse=True)
#sorted_model3_snps = sorted(model3_snps_dict, key=lambda x: model3_snps_dict[x], reverse=True)

In [40]:
combined_snps_dict = {}
def get_top_snps():
    #top_model1 = sorted_model1_snps[:threshold]
    #top_model2 = sorted_model2_snps[:threshold]
    #top_model3 = sorted_model3_snps[:threshold]
    for snp in model1_snps_dict:
        if (snp in model2_snps_dict) and (snp in model3_snps_dict):
            sum_of_weights = model1_snps_dict[snp] + model2_snps_dict[snp] + model3_snps_dict[snp]
            combined_snps_dict[snp] = sum_of_weights
    return combined_snps_dict

In [41]:
tstart = datetime.datetime.now()
combined_snps_dict = get_top_snps()
tfinish = datetime.datetime.now()
texecute = tfinish - tstart
print(texecute)

0:00:00.836000


In [45]:
combined_snps_dict

{'rs12565286_C': 1.6118345486937252e-06,
 'rs12082473_A': 8.6626520747232777e-06,
 'rs3094315_G': 2.4509286161446094e-05,
 'rs2286139_C': 3.35638474280159e-05,
 'rs11240776_G': 1.8300765882686427e-06,
 'rs2980319_A': 2.1741417419628536e-05,
 'rs2980300_T': 3.3597262831342475e-05,
 'rs2905036_C': 3.5771800792479965e-06,
 'rs11240777_A': 2.8169075755688527e-05,
 'rs4245756_T': 3.9713098449504863e-06,
 'rs3748597_T': 3.2168159740895813e-06,
 'rs2341354_A': 2.3255434204235494e-05,
 'rs4970403_A': 1.2324834933161264e-05,
 'rs2465126_G': 2.2268675474165497e-05,
 'rs1921_A': 9.9693207938000598e-06,
 'rs2465136_C': 1.6671688034197277e-05,
 'rs2710872_C': 6.9097958777145016e-06,
 'rs13129_T': 6.3830840826424749e-06,
 'rs4075116_C': 1.7711819676143312e-05,
 'rs6667248_G': 1.0382284030696132e-05,
 'rs9442372_A': 1.1277211535502176e-05,
 'rs11260595_A': 4.3846291655253401e-06,
 'rs6604968_G': 2.1820528328872274e-05,
 'rs4970405_G': 1.2565743360450912e-05,
 'rs6686003_A': 1.4489635880869711e-05,
 '

In [46]:
sorted_combined_snps = sorted(combined_snps_dict, key=lambda x: combined_snps_dict[x], reverse=True)
top_snps = sorted_combined_snps[:1000] # : n snps from sort

In [47]:
len(top_snps)

1000

In [50]:
fw = open("svm_snps_topK.txt","w")
for snp in top_snps:
    fw.write(snp)
    fw.write('\t')
    fw.write(str(combined_snps_dict[snp]))
    fw.write('\n')
fw.close()

In [ ]:
top_snps = get_top_snps(150000)
len(top_snps)

In [ ]:
fw = open("svm_snps_top150k.txt","w")
for snp in top_snps:
    fw.write(snp)
    fw.write('\n')
fw.close()